## 공공데이터 포털
- 병원 / 의원 정보 수집

In [16]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import requests
from bs4 import BeautifulSoup
import glob, os
import urllib
from urllib.parse import urlencode, quote_plus
import requests
from bs4 import BeautifulSoup
from pandas import json_normalize
import xml.dom.minidom

### 약국주소 가져오기

In [17]:
def 약국매핑(지역):
    ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
    open_url = 'http://apis.data.go.kr/B552657/ErmctInsttInfoInqireService/getParmacyListInfoInqire'

    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                                    quote_plus('Q0') : 지역,
                                   quote_plus('pageNo') : '1',
                                   quote_plus('numOfRows') : '10000' })
    open_url_get = open_url + queryParams

    res = requests.get(open_url_get)
    datas = BeautifulSoup(res.content,"html.parser").find_all("item")
    기관주소 = [] ; 기관명 = [] ; 경도 = [] ; 위도 = []

    for data in datas:
        기관주소.append(data.find("dutyaddr").get_text())
        기관명.append(data.find("dutyname").get_text())
        try:
            경도.append(data.find("wgs84lon").get_text()) 
        except: 경도.append(0)
        try:
            위도.append(data.find("wgs84lat").get_text()) 
        except: 위도.append(0)

    df = pd.DataFrame(columns = ["기관주소","기관명","경도","위도"])

    df["기관주소"] = 기관주소
    df["기관명"] = 기관명
    df["경도"] = 경도
    df["위도"] = 위도
    return df

In [18]:
약국_수원 = 약국매핑('수원')
약국_진주 = 약국매핑('진주시')
약국_창원 = 약국매핑('창원시')

서울 약국 가져오기

In [19]:
약국_서울 = pd.read_csv("서울특별시_약국_정보.csv",encoding="cp949")[["주소","약국명","병원경도","병원위도"]]
약국_서울.columns = ["기관주소","기관명","경도","위도"]

In [20]:
약국_4개시 = pd.concat([약국_수원,약국_서울,약국_진주,약국_창원]).reset_index(drop=True)
약국_4개시.columns = ["기관주소","기관명","lon","lat"]
약국_4개시 = 약국_4개시[약국_4개시["lat"].notnull()]

In [22]:
약국_4개시.to_csv("4개시도_약국.csv",encoding="utf-8-sig")

### 병원 주소 가져오기

In [31]:
def 병원매핑(지역):
    ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
    open_url = 'http://apis.data.go.kr/B552657/HsptlAsembySearchService/getHsptlMdcncListInfoInqire'

    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                                    quote_plus('Q0') : 지역,
                                   quote_plus('pageNo') : '1',
                                   quote_plus('numOfRows') : '10000' })
    open_url_get = open_url + queryParams

    res = requests.get(open_url_get)
    datas = BeautifulSoup(res.content,"html.parser").find_all("item")
    기관주소 = [] ; 기관명 = [] ; 경도 = [] ; 위도 = []

    for data in datas:
        기관주소.append(data.find("dutyaddr").get_text())
        기관명.append(data.find("dutyname").get_text())
        try:
            경도.append(data.find("wgs84lon").get_text()) 
        except: 경도.append(0)
        try:
            위도.append(data.find("wgs84lat").get_text()) 
        except: 위도.append(0)

    df = pd.DataFrame(columns = ["기관주소","기관명","경도","위도"])

    df["기관주소"] = 기관주소
    df["기관명"] = 기관명
    df["경도"] = 경도
    df["위도"] = 위도
    return df

In [32]:
병원_수원 = 병원매핑('수원')
#병원_서울 = 병원매핑('서울특별시')
병원_진주 = 병원매핑('진주시')
병원_창원 = 병원매핑('창원시')

In [36]:
병원_서울 = pd.read_csv("서울특별시_병의원_정보.csv",encoding="cp949")[["주소","기관명","병원경도","병원위도"]]
병원_서울.columns = ["기관주소","기관명","경도","위도"]

In [37]:
병원_4개시 = pd.concat([병원_수원,병원_서울,병원_진주,병원_창원])
병원_4개시.columns = ["기관주소","기관명","lon","lat"]
병원_4개시 = 병원_4개시[병원_4개시["lat"].notnull()]

In [39]:
병원_4개시.to_csv("4개시도_병원.csv",encoding="utf-8-sig")